In [262]:
# Initialize Otter
import otter
grader = otter.Notebook("lab4.ipynb")

In [263]:
# Run command for helper functions
%run -i ./helpers/helper_functions.py

In [264]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.feature_selection import RFECV

import nltk 
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import altair as alt
import altair_ally as aly

alt.data_transformers.enable("vegafusion")


DataTransformerRegistry.enable('vegafusion')

# Lab 4: Putting it all together in a mini project

For this lab, **you can choose to work alone of in a group of up to four students**. You are in charge of how you want to work and who you want to work with. Maybe you really want to go through all the steps of the ML process yourself or maybe you want to practice your collaboration skills, it is up to you! Just remember to indicate who your group members are (if any) when you submit on Gradescope. If you choose to work in a group, you only need to use one GitHub repo (you can create one on github.ubc.ca and set the visibility to "public"). If it takes a prohibitively long time to run any of the steps on your laptop, it is OK if you sample the data to reduce the runtime, just make sure you write a note about this.

<!-- BEGIN QUESTION -->

<div class="alert alert-info">

## Submission instructions
rubric={mechanics}

<p>You receive marks for submitting your lab correctly, please follow these instructions:</p>

<ul>
  <li><a href="https://ubc-mds.github.io/resources_pages/general_lab_instructions/">
      Follow the general lab instructions.</a></li>
  <li><a href="https://github.com/UBC-MDS/public/tree/master/rubric">
      Click here to view a description of the rubrics used to grade the questions</a></li>
  <li>Make at least three commits.</li>
  <li>Push your <code>.ipynb</code> file to your GitHub repository for this lab and upload it to Gradescope.</li>
    <ul>
      <li>Before submitting, make sure you restart the kernel and rerun all cells.</li>
    </ul>
  <li>Also upload a <code>.pdf</code> export of the notebook to facilitate grading of manual questions (preferably WebPDF, you can select two files when uploading to gradescope)</li>
  <li>Don't change any variable names that are given to you, don't move cells around, and don't include any code to install packages in the notebook.</li>
  <li>The data you download for this lab <b>SHOULD NOT BE PUSHED TO YOUR REPOSITORY</b> (there is also a <code>.gitignore</code> in the repo to prevent this).</li>
  <li>Include a clickable link to your GitHub repo for the lab just below this cell
    <ul>
      <li>It should look something like this https://github.ubc.ca/MDS-2020-21/DSCI_531_labX_yourcwl.</li>
    </ul>
  </li>
</ul>
</div>

_Points:_ 2

[TEAM BDFJ GitHub URL](https://github.ubc.ca/mds-2024-25/DSCI_573_Lab4_BDFJ)

<!-- END QUESTION -->

## Introduction <a name="in"></a>

In this lab you will be working on an open-ended mini-project, where you will put all the different things you have learned so far in 571 and 573 together to solve an interesting problem.

A few notes and tips when you work on this mini-project: 

#### Tips
1. Since this mini-project is open-ended there might be some situations where you'll have to use your own judgment and make your own decisions (as you would be doing when you work as a data scientist). Make sure you explain your decisions whenever necessary. 
2. **Do not include everything you ever tried in your submission** -- it's fine just to have your final code. That said, your code should be reproducible and well-documented. For example, if you chose your hyperparameters based on some hyperparameter optimization experiment, you should leave in the code for that experiment so that someone else could re-run it and obtain the same hyperparameters, rather than mysteriously just setting the hyperparameters to some (carefully chosen) values in your code. 
3. If you realize that you are repeating a lot of code try to organize it in functions. Clear presentation of your code, experiments, and results is the key to be successful in this lab. You may use code from lecture notes or previous lab solutions with appropriate attributions. 

#### Assessment
We don't have some secret target score that you need to achieve to get a good grade. **You'll be assessed on demonstration of mastery of course topics, clear presentation, and the quality of your analysis and results.** For example, if you just have a bunch of code and no text or figures, that's not good. If you instead do a bunch of sane things and you have clearly motivated your choices, but still get lower model performance than your friend, don't sweat it.


#### A final note
Finally, the style of this "project" question is different from other assignments. It'll be up to you to decide when you're "done" -- in fact, this is one of the hardest parts of real projects. But please don't spend WAY too much time on this... perhaps "several hours" but not "many hours" is a good guideline for a high quality submission. Of course if you're having fun you're welcome to spend as much time as you want! But, if so, try not to do it out of perfectionism or getting the best possible grade. Do it because you're learning and enjoying it. Students from the past cohorts have found such kind of labs useful and fun and we hope you enjoy it as well. 

<!-- BEGIN QUESTION -->

<div class="alert alert-info">

## 1. Pick your problem and explain the prediction problem <a name="1"></a>
rubric={reasoning}

In this mini project, you will pick one of the following problems: 

1. A classification problem of predicting whether a credit card client will default or not. For this problem, you will use [Default of Credit Card Clients Dataset](https://www.kaggle.com/uciml/default-of-credit-card-clients-dataset). In this data set, there are 30,000 examples and 24 features, and the goal is to estimate whether a person will default (fail to pay) their credit card bills; this column is labeled "default.payment.next.month" in the data. The rest of the columns can be used as features. You may take some ideas and compare your results with [the associated research paper](https://www.sciencedirect.com/science/article/pii/S0957417407006719), which is available through [the UBC library](https://www.library.ubc.ca/). 

OR 

2. A regression problem of predicting `reviews_per_month`, as a proxy for the popularity of the listing with [New York City Airbnb listings from 2019 dataset](https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data). Airbnb could use this sort of model to predict how popular future listings might be before they are posted, perhaps to help guide hosts create more appealing listings. In reality they might instead use something like vacancy rate or average rating as their target, but we do not have that available here.

**Your tasks:**

1. Spend some time understanding the problem and what each feature means. Write a few sentences on your initial thoughts on the problem and the dataset. 
2. Download the dataset and read it as a pandas dataframe. 
3. Carry out any preliminary preprocessing, if needed (e.g., changing feature names, handling of NaN values etc.)
    
</div>

_Points:_ 3

_Type your answer here, replacing this text._

In [265]:
data = pd.read_csv('./data/raw/AB_NYC_2019.csv')
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<div class="alert alert-info">

## 2. Data splitting <a name="2"></a>
rubric={reasoning}

**Your tasks:**

1. Split the data into train and test portions.

> Make the decision on the `test_size` based on the capacity of your laptop. 
    
</div>

_Points:_ 1

In [266]:
# Checking if all null target values is because the number of reviews is 0
data[(data['number_of_reviews'] != 0) & (data['reviews_per_month'].isnull())]

# Fix target null values to be 0 when number of reviews is 0.
data['reviews_per_month'] = data['reviews_per_month'].apply(lambda x: 0 if pd.isna(x) else x)

train_df, test_df = train_test_split(data, random_state=573)

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<div class="alert alert-info">

## 3. EDA <a name="3"></a>
rubric={viz,reasoning}
    
Perform exploratory data analysis on the train set.

**Your tasks:**

1. Include at least two summary statistics and two visualizations that you find useful, and accompany each one with a sentence explaining it.
2. Summarize your initial observations about the data. 
3. Pick appropriate metric/metrics for assessment. 
    
</div>

_Points:_ 6

_Type your answer here, replacing this text._

In [267]:
train_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
4888,3495686,Lofted Murray Hill 1-bedroom w/ spiral staircase,721833,Joshua,Manhattan,Kips Bay,40.74593,-73.97915,Entire home/apt,184,3,0,NaN,0.00,1,0
42318,32843454,"Cute, Vintage Studio on UES Close to Express Q",247072396,Andrea,Manhattan,Upper East Side,40.77085,-73.94933,Entire home/apt,250,2,1,2019-04-21,0.38,1,0
30859,23896110,Charming one bedroom / West village,175315624,Shelby,Manhattan,West Village,40.73170,-74.00708,Entire home/apt,163,7,7,2018-07-18,0.48,1,0
38670,30379843,"Your NY home for Quality time , Fully Equipped !",148725429,Javier,Brooklyn,Bushwick,40.69231,-73.92567,Entire home/apt,90,3,58,2019-07-07,7.91,2,2
40083,31119351,Beautiful Brooklyn Home,2557889,Serena,Brooklyn,Bushwick,40.70267,-73.92751,Private room,55,2,16,2019-05-13,2.57,1,2


In [268]:
train_df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,3.667100e+04,3.667100e+04,36671.000000,36671.000000,36671.000000,36671.000000,36671.000000,36671.000000,36671.000000,36671.000000
mean,1.901858e+07,6.753485e+07,40.728871,-73.952174,153.009408,7.017398,23.086990,1.086294,7.155109,112.255897
std,1.099541e+07,7.862213e+07,0.054648,0.046113,247.269517,20.734663,44.304601,1.593990,33.242306,131.558583
min,3.647000e+03,2.438000e+03,40.499790,-74.244420,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000
25%,9.453066e+06,7.848642e+06,40.689930,-73.983090,69.000000,1.000000,1.000000,0.040000,1.000000,0.000000
50%,1.968643e+07,3.091859e+07,40.722830,-73.955680,105.000000,2.000000,5.000000,0.370000,1.000000,44.000000
75%,2.917369e+07,1.074344e+08,40.763125,-73.936160,175.000000,5.000000,23.000000,1.570000,2.000000,225.000000
max,3.648543e+07,2.743213e+08,40.912340,-73.712990,10000.000000,1250.000000,629.000000,58.500000,327.000000,365.000000


In [269]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36671 entries, 4888 to 9822
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              36671 non-null  int64  
 1   name                            36658 non-null  object 
 2   host_id                         36671 non-null  int64  
 3   host_name                       36656 non-null  object 
 4   neighbourhood_group             36671 non-null  object 
 5   neighbourhood                   36671 non-null  object 
 6   latitude                        36671 non-null  float64
 7   longitude                       36671 non-null  float64
 8   room_type                       36671 non-null  object 
 9   price                           36671 non-null  int64  
 10  minimum_nights                  36671 non-null  int64  
 11  number_of_reviews               36671 non-null  int64  
 12  last_review                     291

In [270]:
col_list = ['price','availability_365','number_of_reviews','minimum_nights']

chart = alt.Chart(data).mark_bar().encode(
    x=alt.X(alt.repeat('column')).bin(),
    y='count()'
).repeat(
    column=col_list
)
chart

alt.RepeatChart(...)

In [271]:
aly.corr(train_df)

alt.ConcatChart(...)

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<div class="alert alert-warning">

## 4. Feature engineering (Challenging)
rubric={reasoning}

**Your tasks:**

1. Carry out feature engineering. In other words, extract new features relevant for the problem and work with your new feature set in the following exercises. You may have to go back and forth between feature engineering and preprocessing.
    
</div>

_Points:_ 0.5

In [272]:
nltk.download('vader_lexicon')
nltk.download('punkt')

sid = SentimentIntensityAnalyzer()

train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36671 entries, 4888 to 9822
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              36671 non-null  int64  
 1   name                            36658 non-null  object 
 2   host_id                         36671 non-null  int64  
 3   host_name                       36656 non-null  object 
 4   neighbourhood_group             36671 non-null  object 
 5   neighbourhood                   36671 non-null  object 
 6   latitude                        36671 non-null  float64
 7   longitude                       36671 non-null  float64
 8   room_type                       36671 non-null  object 
 9   price                           36671 non-null  int64  
 10  minimum_nights                  36671 non-null  int64  
 11  number_of_reviews               36671 non-null  int64  
 12  last_review                     291

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/brianchang/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/brianchang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [273]:
# Add polarity score data column
train_df['name_polarity_scores'] = train_df['name'].apply(lambda x: None if pd.isna(x) else sid.polarity_scores(x)['compound'])

train_df.head(3)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,name_polarity_scores
4888,3495686,Lofted Murray Hill 1-bedroom w/ spiral staircase,721833,Joshua,Manhattan,Kips Bay,40.74593,-73.97915,Entire home/apt,184,3,0,NaN,0.00,1,0,0.0000
42318,32843454,"Cute, Vintage Studio on UES Close to Express Q",247072396,Andrea,Manhattan,Upper East Side,40.77085,-73.94933,Entire home/apt,250,2,1,2019-04-21,0.38,1,0,0.4588
30859,23896110,Charming one bedroom / West village,175315624,Shelby,Manhattan,West Village,40.73170,-74.00708,Entire home/apt,163,7,7,2018-07-18,0.48,1,0,0.5859


In [274]:
train_df['month_of_last_review'] = train_df['last_review'].apply(lambda x: x if pd.isna(x) else x.split('-')[1])

num_to_month_map = {
    '01': "January", '02': "February", '03': "March", '04': "April",
    '05': "May", '06': "June", '07': "July", '08': "August",
    '09': "September", '10': "October", '11': "November", '12': "December"
}

train_df['month_of_last_review'] = train_df['month_of_last_review'].apply((lambda x: x if pd.isna(x) else num_to_month_map[x]))

train_df.head(3)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,name_polarity_scores,month_of_last_review
4888,3495686,Lofted Murray Hill 1-bedroom w/ spiral staircase,721833,Joshua,Manhattan,Kips Bay,40.74593,-73.97915,Entire home/apt,184,3,0,NaN,0.00,1,0,0.0000,NaN
42318,32843454,"Cute, Vintage Studio on UES Close to Express Q",247072396,Andrea,Manhattan,Upper East Side,40.77085,-73.94933,Entire home/apt,250,2,1,2019-04-21,0.38,1,0,0.4588,April
30859,23896110,Charming one bedroom / West village,175315624,Shelby,Manhattan,West Village,40.73170,-74.00708,Entire home/apt,163,7,7,2018-07-18,0.48,1,0,0.5859,July


<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<div class="alert alert-info">

## 5. Preprocessing and transformations <a name="5"></a>
rubric={accuracy,reasoning}

**Your tasks:**

1. Identify different feature types and the transformations you would apply on each feature type. 
2. Define a column transformer, if necessary. 
    
</div>

_Points:_ 4

In [275]:
X_train = train_df.drop(columns='reviews_per_month')
y_train = train_df['reviews_per_month']
X_test = test_df.drop(columns='reviews_per_month')
y_test = test_df['reviews_per_month']

In [276]:
categorical_features = ['neighbourhood_group', 'room_type',]
numeric_features = ['latitude', 
                'longitude', 
                'minimum_nights', 
                'number_of_reviews', 
                'calculated_host_listings_count', 
                'availability_365'
                ]
drop_features = ['id', 'host_id', 'host_name', 'name', 'neighbourhood']

categorical_transformer = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="missing"),
    OneHotEncoder(handle_unknown="ignore", sparse_output=False),
)

numeric_transformer = make_pipeline(
    SimpleImputer(strategy="median"), 
    StandardScaler()
)

preprocessor = make_column_transformer(
    (numeric_transformer, numeric_features),
    (categorical_transformer, categorical_features),
    ("drop", drop_features)
)

In [277]:
preprocessor.fit(X_train) 

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['latitude', 'longitude', 'minimum_nights',
                                  'number_of_reviews',
                                  'calculated_host_listings_count',
                                  'availability_365']),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['neighbourhood_group', 'room_type']),
                                ('drop', 'drop',
                                 ['id', 'host_id', 'host_name', 'name',
                                  'neighbourhood'])])

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<div class="alert alert-info">

## 6. Baseline model <a name="6"></a>
rubric={accuracy}

**Your tasks:**
1. Train a baseline model for your task and report its performance.
    
</div>

_Points:_ 2

In [278]:
dummy = DummyRegressor()
scoring_metric = {
    "R2": "r2",
    "Neg RMSE": "neg_root_mean_squared_error",
}
results = {}

results["Dummy"] = mean_std_cross_val_scores(
    dummy, X_train, y_train, return_train_score=True, scoring=scoring_metric
)
results["Dummy"]


fit_time           0.001 (+/- 0.000)
score_time         0.000 (+/- 0.000)
test_R2           -0.000 (+/- 0.000)
train_R2           0.000 (+/- 0.000)
test_Neg RMSE     -1.593 (+/- 0.065)
train_Neg RMSE    -1.594 (+/- 0.017)
dtype: object

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<div class="alert alert-info">

## 7. Linear models <a name="7"></a>
rubric={accuracy,reasoning}

**Your tasks:**

1. Try a linear model as a first real attempt. 
2. Carry out hyperparameter tuning to explore different values for the regularization hyperparameter. 
3. Report cross-validation scores along with standard deviation. 
4. Summarize your results.
    
</div>

_Points:_ 8

_Type your answer here, replacing this text._

In [279]:
alpha_vals =  np.logspace(-8, 8, 10)

ridge = make_pipeline(
    preprocessor,
    PolynomialFeatures(degree=2),
    RidgeCV(alphas = alpha_vals, cv=10)
)

results["RidgeCV"] = mean_std_cross_val_scores(
    ridge, X_train, y_train, return_train_score=True, scoring=scoring_metric
)
results["RidgeCV"]

/Users/brianchang/miniforge3/envs/573/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.05978e-16): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/brianchang/miniforge3/envs/573/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.08576e-16): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/brianchang/miniforge3/envs/573/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.09152e-16): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/brianchang/miniforge3/envs/573/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=9.06393e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", 

fit_time           0.979 (+/- 0.006)
score_time         0.005 (+/- 0.000)
test_R2            0.438 (+/- 0.029)
train_R2           0.449 (+/- 0.007)
test_Neg RMSE     -1.194 (+/- 0.075)
train_Neg RMSE    -1.183 (+/- 0.019)
dtype: object

In [280]:
...

Ellipsis

In [281]:
...

Ellipsis

In [282]:
...

Ellipsis

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<div class="alert alert-info">

## 8. Different models <a name="8"></a>
rubric={accuracy,reasoning}

**Your tasks:**
1. Try out three other models aside from the linear model. 
2. Summarize your results in terms of overfitting/underfitting and fit and score times. Can you beat the performance of the linear model? 
    
</div>

_Points:_ 10

_Type your answer here, replacing this text._

In [283]:
# LGBM
from lightgbm.sklearn import LGBMRegressor
pipe_lgbm = make_pipeline(
    preprocessor,
    PolynomialFeatures(degree=2),
    LGBMRegressor(
        n_jobs=-1,
        random_state=573
    )
)

results['lgbm'] = mean_std_cross_val_scores(
    pipe_lgbm, X_train, y_train, return_train_score=True, scoring=scoring_metric
)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001241 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12964
[LightGBM] [Info] Number of data points in the train set: 29336, number of used features: 105
[LightGBM] [Info] Start training from score 1.086381
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001192 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12964
[LightGBM] [Info] Number of data points in the train set: 29337, number of used features: 105
[LightGBM] [Info] Start training from score 1.086454
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001184 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12928
[LightGBM] [Info] Number of data points in the train set: 29337, number of used features: 105
[LightGBM] [Info] Sta

In [284]:
# Random Forest
# Code adapted from Lecture 7: https://pages.github.ubc.ca/mds-2024-25/DSCI_573_feat-model-select_students/lectures/07_ensembles.html#random-forests
from sklearn.ensemble import RandomForestRegressor

pipe_rf = make_pipeline(
    preprocessor,
    PolynomialFeatures(degree=2),
    RandomForestRegressor(
        n_jobs=-1,
        random_state=573,
    )
)

results["random_forests"] = mean_std_cross_val_scores(
    pipe_rf, X_train, y_train, return_train_score=True, scoring=scoring_metric
)


In [285]:
# ElasticNet
from sklearn.linear_model import ElasticNetCV
# 1. Setup the ElasticNet Pipeline
elastic = make_pipeline(
    preprocessor,
    PolynomialFeatures(degree=2),
    ElasticNetCV(max_iter=20_000, tol=0.01, cv=10)
)

# 2. Cross-validation
results['elastic_net'] = mean_std_cross_val_scores(
    elastic, X_train, y_train, return_train_score=True, scoring=scoring_metric, cv=10
)

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<div class="alert alert-warning">

## 9. Feature selection (Challenging)
rubric={reasoning}

**Your tasks:**

Make some attempts to select relevant features. You may try `RFECV`, forward/backward selection or L1 regularization for this. Do the results improve with feature selection? Summarize your results. If you see improvements in the results, keep feature selection in your pipeline. If not, you may abandon it in the next exercises unless you think there are other benefits with using less features.
    
</div>

_Points:_ 0.5

_Type your answer here, replacing this text._

In [286]:
# Make the rfecv pipeline
pipe_rfe_ridgecv = make_pipeline(
    preprocessor,
    PolynomialFeatures(degree=2),
    RFECV(Ridge(), cv=10, n_jobs=-1), 
    RidgeCV()
)


# Get the cv scores
results['rfe_ridgecv'] = mean_std_cross_val_scores(
    pipe_rfe_ridgecv, X_train, y_train, return_train_score=True, scoring=scoring_metric
)

In [287]:
pd.DataFrame(results).T

,fit_time,score_time,test_R2,train_R2,test_Neg RMSE,train_Neg RMSE
Dummy,0.001 (+/- 0.000),0.000 (+/- 0.000),-0.000 (+/- 0.000),0.000 (+/- 0.000),-1.593 (+/- 0.065),-1.594 (+/- 0.017)
RidgeCV,0.979 (+/- 0.006),0.005 (+/- 0.000),0.438 (+/- 0.029),0.449 (+/- 0.007),-1.194 (+/- 0.075),-1.183 (+/- 0.019)
lgbm,0.664 (+/- 0.007),0.008 (+/- 0.000),0.599 (+/- 0.033),0.701 (+/- 0.006),-1.009 (+/- 0.081),-0.871 (+/- 0.017)
random_forests,4.564 (+/- 0.068),0.033 (+/- 0.000),0.581 (+/- 0.033),0.941 (+/- 0.002),-1.031 (+/- 0.082),-0.387 (+/- 0.010)
elastic_net,1.056 (+/- 0.195),0.004 (+/- 0.000),0.444 (+/- 0.038),0.445 (+/- 0.006),-1.187 (+/- 0.113),-1.188 (+/- 0.015)
rfe_ridgecv,2.555 (+/- 0.904),0.007 (+/- 0.001),0.438 (+/- 0.028),0.450 (+/- 0.008),-1.194 (+/- 0.073),-1.182 (+/- 0.020)


<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<div class="alert alert-info">

## 10. Hyperparameter optimization
rubric={accuracy,reasoning}

**Your tasks:**

Make some attempts to optimize hyperparameters for the models you've tried and summarize your results. In at least one case you should be optimizing multiple hyperparameters for a single model. You may use `sklearn`'s methods for hyperparameter optimization or fancier Bayesian optimization methods.  Briefly summarize your results.
  - [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)   
  - [RandomizedSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)
  - [scikit-optimize](https://github.com/scikit-optimize/scikit-optimize) 
    
</div>

_Points:_ 6

_Type your answer here, replacing this text._

In [292]:
from scipy.stats import loguniform

param_dist_ridge = {"ridgecv__alphas": loguniform(1e-3, 1e3)}

random_ridge_search = RandomizedSearchCV(
    pipe_rfe_ridgecv, # using the rfecv pipe earlier that we already created
    param_distributions=param_dist_ridge, 
    n_iter=10,
    verbose=1,
    n_jobs=-1,
    random_state=573,
    return_train_score=True,
    scoring=scoring_metric, 
    refit='R2'
)

In [293]:
random_ridge_search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/Users/brianchang/miniforge3/envs/573/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
4 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/Users/brianchang/miniforge3/envs/573/lib/python3.12/site-packages/joblib/_utils.py", line 72, in __call__
    return self.func(**kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/Users/brianchang/miniforge3/envs/573/lib/python3.12/site-packages/joblib/parallel.py", line 598, in __call__
    return [func(*args, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/brianchang/minifor

RandomizedSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('pipeline-1',
                                                                               Pipeline(steps=[('simpleimputer',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('standardscaler',
                                                                                                StandardScaler())]),
                                                                               ['latitude',
                                                                                'longitude',
                                                                                'minimum_nights',
                                                                                'number_of_reviews',
                                                                                'calculated_host_listings_count',
                                                                                'availability_365']),
                                                                              ('pipeline-2',
                                                                               Pipe...
                                             ('polynomialfeatures',
                                              PolynomialFeatures()),
                                             ('rfecv',
                                              RFECV(cv=10, estimator=Ridge(),
                                                    n_jobs=-1)),
                                             ('ridgecv', RidgeCV())]),
                   n_jobs=-1,
                   param_distributions={'ridgecv__alphas': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x380751a60>},
                   random_state=573, refit='R2', return_train_score=True,
                   scoring={'Neg RMSE': 'neg_root_mean_squared_error',
                            'R2': 'r2'},
                   verbose=1)

In [294]:
random_ridge_search.best_params_

{'ridgecv__alphas': np.float64(0.213968041635095)}

In [295]:
opti_results = {}


opti_results['ridge'] = random_ridge_search.best_score_

In [296]:
pipe_lgbm_reduced = make_pipeline(
    preprocessor,
    PolynomialFeatures(degree=2),
    RFECV(Ridge(), cv=5, n_jobs=-1), 
    LGBMRegressor(
        n_jobs=-1,
        random_state=573
    )
)

param_grid_lgbm =  {
    "lgbmregressor__learning_rate": [0.01, 0.05, 0.1, 0.15, 0.2],
    "lgbmregressor__max_depth": [-1, 10,50,100],
    "lgbmregressor__n_estimators": [100,150,200]
}

In [297]:
random_lgbm_search = RandomizedSearchCV(
    pipe_lgbm_reduced,
    param_distributions=param_grid_lgbm,
    n_iter=10,
    n_jobs=-1,
    verbose=1,
    return_train_score=True,
    scoring=scoring_metric,
    refit='R2'
)

In [298]:
random_lgbm_search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017069 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12964
[LightGBM] [Info] Number of data points in the train set: 29336, number of used features: 105
[LightGBM] [Info] Start training from score 1.086381
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009049 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12687
[LightGBM] [Info] Number of data points in the train set: 29337, number of used features: 101
[LightGBM] [Info] Start training from score 1.086966
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.227210 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12673
[LightGBM] [Info] Auto-choosing col-wise multi-threadi

/Users/brianchang/miniforge3/envs/573/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005982 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12687
[LightGBM] [Info] Number of data points in the train set: 29337, number of used features: 101
[LightGBM] [Info] Start training from score 1.086966
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001889 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12964
[LightGBM] [Info] Number of data points in the train set: 29337, number of used features: 105
[LightGBM] [Info] Start training from score 1.086454
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002950 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12673
[LightGBM] [Info] Number of data points in the train set: 29337, number of used features: 104
[LightGBM] [Info] Sta

/Users/brianchang/miniforge3/envs/573/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
1 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/Users/brianchang/miniforge3/envs/573/lib/python3.12/site-packages/joblib/_utils.py", line 72, in __call__
    return self.func(**kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/Users/brianchang/miniforge3/envs/573/lib/python3.12/site-packages/joblib/parallel.py", line 598, in __call__
    return [func(*args, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/brianchang/minifor

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001569 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13217
[LightGBM] [Info] Number of data points in the train set: 36671, number of used features: 104
[LightGBM] [Info] Start training from score 1.086294


RandomizedSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('pipeline-1',
                                                                               Pipeline(steps=[('simpleimputer',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('standardscaler',
                                                                                                StandardScaler())]),
                                                                               ['latitude',
                                                                                'longitude',
                                                                                'minimum_nights',
                                                                                'number_of_reviews',
                                                                                'calculated_host_listings_count',
                                                                                'availability_365']),
                                                                              ('pipeline-2',
                                                                               Pipe...
                                                    n_jobs=-1)),
                                             ('lgbmregressor',
                                              LGBMRegressor(n_jobs=-1,
                                                            random_state=573))]),
                   n_jobs=-1,
                   param_distributions={'lgbmregressor__learning_rate': [0.01,
                                                                         0.05,
                                                                         0.1,
                                                                         0.15,
                                                                         0.2],
                                        'lgbmregressor__max_depth': [-1, 10, 50,
                                                                     100],
                                        'lgbmregressor__n_estimators': [100,
                                                                        150,
                                                                        200]},
                   refit='R2', return_train_score=True,
                   scoring={'Neg RMSE': 'neg_root_mean_squared_error',
                            'R2': 'r2'},
                   verbose=1)

In [299]:
random_lgbm_search.best_params_

{'lgbmregressor__n_estimators': 200,
 'lgbmregressor__max_depth': 100,
 'lgbmregressor__learning_rate': 0.1}

In [300]:
opti_results['lgbm'] = random_lgbm_search.best_score_

In [303]:
# Random Forest Regresser Optimization
pipe_rf_reduced = make_pipeline(
    preprocessor,
    PolynomialFeatures(degree=2),
    RFECV(Ridge(), cv=5, n_jobs=-1), 
    RandomForestRegressor(
        n_jobs=-1,
        random_state=573,
    )
)

param_grid_rf = {
    "randomforestregressor__max_depth": np.linspace(2, 20, 10, dtype=int),
    "randomforestregressor__n_estimators": np.linspace(10, 500, 20, dtype=int)
}


random_rf_search = RandomizedSearchCV(
    pipe_rf_reduced,                                    
    param_distributions = param_grid_rf,
    n_iter=10,
    verbose=1,
    n_jobs=-1,
    random_state=573,
    return_train_score=True,
    scoring=scoring_metric, 
    refit='R2'
)

In [304]:
random_rf_search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('pipeline-1',
                                                                               Pipeline(steps=[('simpleimputer',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('standardscaler',
                                                                                                StandardScaler())]),
                                                                               ['latitude',
                                                                                'longitude',
                                                                                'minimum_nights',
                                                                                'number_of_reviews',
                                                                                'calculated_host_listings_count',
                                                                                'availability_365']),
                                                                              ('pipeline-2',
                                                                               Pipe...
                                                                    random_state=573))]),
                   n_jobs=-1,
                   param_distributions={'randomforestregressor__max_depth': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20]),
                                        'randomforestregressor__n_estimators': array([ 10,  35,  61,  87, 113, 138, 164, 190, 216, 242, 267, 293, 319,
       345, 371, 396, 422, 448, 474, 500])},
                   random_state=573, refit='R2', return_train_score=True,
                   scoring={'Neg RMSE': 'neg_root_mean_squared_error',
                            'R2': 'r2'},
                   verbose=1)

In [305]:
random_rf_search.best_params_

{'randomforestregressor__n_estimators': np.int64(422),
 'randomforestregressor__max_depth': np.int64(16)}

In [306]:
opti_results['rf'] = random_rf_search.best_score_

In [317]:
import pickle
with open('./models/ridge_random.pickle', 'wb') as file:
    pickle.dump(random_ridge_search, file)

with open('./models/lgbm_random.pickle', 'wb') as file:
    pickle.dump(random_lgbm_search, file)

with open('./models/rf_random.pickle', 'wb') as file:
    pickle.dump(random_rf_search, file)
    

In [313]:
# Elastic Net Optimization
pipe_elasticnet_reduced = make_pipeline(
    preprocessor,
    PolynomialFeatures(degree=2),
    RFECV(Ridge(), cv=5, n_jobs=-1), 
    ElasticNetCV(max_iter=20_000, tol=0.01, cv=5)
)

param_grid_elastic = {
    "elasticnetcv__l1_ratio": np.linspace(0.1, 1.0, 10),
    "elasticnetcv__alphas": [0.1, 1.0, 10.0, 100.0],
}

random_en_search = RandomizedSearchCV(
    pipe_elasticnet_reduced, 
    param_distributions=param_grid_elastic,
    n_iter=10,
    verbose=1,
    n_jobs=-1,
    random_state=573,
    return_train_score=True,
    scoring='r2',
    refit='r2'
)

In [314]:
random_en_search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


ValueError: 
All the 50 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/brianchang/miniforge3/envs/573/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/brianchang/miniforge3/envs/573/lib/python3.12/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/brianchang/miniforge3/envs/573/lib/python3.12/site-packages/sklearn/pipeline.py", line 473, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "/Users/brianchang/miniforge3/envs/573/lib/python3.12

In [ ]:
random_en_search.best_params_

Ellipsis

In [ ]:
opti_results['elastic_net'] = random_rf_search.best_score_

In [ ]:
pd.DataFrame(opti_results).T

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<div class="alert alert-info">

## 11. Interpretation and feature importances <a name="1"></a>
rubric={accuracy,reasoning}

**Your tasks:**

1. Use the methods we saw in class (e.g., `permutation_importance` or `shap`) (or any other methods of your choice) to examine the most important features of one of the non-linear models. 
2. Summarize your observations. 
    
</div>

_Points:_ 8

_Type your answer here, replacing this text._

In [ ]:
...

Ellipsis

In [ ]:
...

Ellipsis

In [ ]:
...

Ellipsis

In [ ]:
...

Ellipsis

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<div class="alert alert-info">

## 12. Results on the test set <a name="12"></a>
rubric={accuracy,reasoning}

**Your tasks:**

1. Try your best performing model on the test data and report test scores. 
2. Do the test scores agree with the validation scores from before? To what extent do you trust your results? Do you think you've had issues with optimization bias? 
3. Take one or two test predictions and explain them with SHAP force plots.  
    
</div>

_Points:_ 6

_Type your answer here, replacing this text._

In [ ]:
...

Ellipsis

In [ ]:
...

Ellipsis

In [ ]:
...

Ellipsis

In [ ]:
...

Ellipsis

In [ ]:
...

Ellipsis

In [ ]:
...

Ellipsis

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<div class="alert alert-info">

## 13. Summary of results <a name="13"></a>
rubric={reasoning}

Imagine that you want to present the summary of these results to your boss and co-workers. 

**Your tasks:**

1. Create a table summarizing important results. 
2. Write concluding remarks.
3. Discuss other ideas that you did not try but could potentially improve the performance/interpretability . 
3. Report your final test score along with the metric you used at the top of this notebook.
    
</div>

_Points:_ 8

_Type your answer here, replacing this text._

In [ ]:
...

Ellipsis

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<div class="alert alert-warning">

## 14. Creating a data analysis pipeline (Challenging)
rubric={reasoning}

**Your tasks:**

- Convert this notebook into scripts to create a reproducible data analysis pipeline with appropriate documentation. Submit your project folder in addition to this notebook on GitHub and briefly comment on your organization in the text box below.
    
</div>

_Points:_ 2

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<div class="alert alert-warning">

## 15. Your takeaway from the course (Challenging)
rubric={reasoning}

**Your tasks:**

What is your biggest takeaway from this course? 
    
</div>

_Points:_ 0.25

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<div class="alert alert-danger" style="color:black">
    
**Restart, run all and export a PDF before submitting**
    
Before submitting,
don't forget to run all cells in your notebook
to make sure there are no errors
and so that the TAs can see your plots on Gradescope.
You can do this by clicking the ▶▶ button
or going to `Kernel -> Restart Kernel and Run All Cells...` in the menu.
This is not only important for MDS,
but a good habit you should get into before ever committing a notebook to GitHub,
so that your collaborators can run it from top to bottom
without issues.
    
After running all the cells,
export a PDF of the notebook (preferably the WebPDF export)
and upload this PDF together with the ipynb file to Gradescope
(you can select two files when uploading to Gradescope)
</div>

---

## Help us improve the labs

The MDS program is continually looking to improve our courses, including lab questions and content. The following optional questions will not affect your grade in any way nor will they be used for anything other than program improvement:

1. Approximately how many hours did you spend working or thinking about this assignment (including lab time)?

#Ans:

2. Do you have any feedback on the lab you be willing to share? For example, any part or question that you particularly liked or disliked?

#Ans: